In [79]:
import itertools
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from lightfm.data import Dataset
from dao import MovieLens100kDao as Dao

In [80]:
dao = Dao()
dao.build()

/Users/komoto_naoki/projects/examine-light-fm/venv/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [157]:
df = dao.df
df = df[df["rating"] == 1]
df = df.drop(columns=["zip_code", "rating", "movie_id"])

print("dataset\n")
df

dataset



,user_id,timestamp,movie_title,release_date,genre,age,gender,occupation
2,196,881251793,"Truth About Cats & Dogs, The (1996)",Apr-1996,"[comedy, romance]",49,M,writer
3,196,881251955,"Birdcage, The (1996)",Mar-1996,[comedy],49,M,writer
4,196,881251843,"Adventures of Priscilla, Queen of the Desert, ...",Jan-1994,"[comedy, drama]",49,M,writer
6,196,881251820,"Fish Called Wanda, A (1988)",Jan-1988,[comedy],49,M,writer
7,196,881250949,"English Patient, The (1996)",Nov-1996,"[drama, romance, war]",49,M,writer
...,...,...,...,...,...,...,...,...
99981,873,891392177,Rosewood (1997),Feb-1997,[drama],48,F,administrator
99984,873,891392238,Air Force One (1997),Jan-1997,"[action, thriller]",48,F,administrator
99985,873,891392177,Titanic (1997),Jan-1997,"[action, drama, romance]",48,F,administrator
99986,873,891392656,G.I. Jane (1997),Jan-1997,"[action, drama, war]",48,F,administrator


In [158]:
# add prefix to values to avoid duplications
df["age"] = "age-" + df["age"].astype(str)
df["gender"] = "gender-" + df["gender"].astype(str)
df["occupation"] = "occupation-" + df["occupation"].astype(str)

df["release_date"] = "release_date-" + df["release_date"].astype(str)
df["genre"] = [["genre-" + value for value in i] for i in df["genre"].values.tolist()]

In [159]:
df["user_features"] = df[["age", "gender", "occupation"]].values.tolist()
df["item_features"] = df[["release_date", "genre"]].values.tolist()

In [160]:
def flatten_sequences(sequences):
    sequences = [i if type(i) == list else [i] for i in sequences]
    flattened = list(itertools.chain.from_iterable(sequences))
    return flattened

In [161]:
df["item_features"] = df["item_features"].apply(flatten_sequences)

In [162]:
df = df[["user_id", "movie_title", "user_features", "item_features", "timestamp"]]
df = df.sort_values(by=["user_id", "timestamp"]).reset_index(drop=True)

In [163]:
df

,user_id,movie_title,user_features,item_features,timestamp
0,1,"Empire Strikes Back, The (1980)","[age-24, gender-M, occupation-technician]","[release_date-Jan-1980, genre-action, genre-ad...",874965478
1,1,Monty Python and the Holy Grail (1974),"[age-24, gender-M, occupation-technician]","[release_date-Jan-1974, genre-comedy]",874965478
2,1,Jean de Florette (1986),"[age-24, gender-M, occupation-technician]","[release_date-Jan-1986, genre-drama]",874965518
3,1,Reservoir Dogs (1992),"[age-24, gender-M, occupation-technician]","[release_date-Jan-1992, genre-crime, genre-thr...",874965556
4,1,Dead Poets Society (1989),"[age-24, gender-M, occupation-technician]","[release_date-Jan-1989, genre-drama]",874965677
...,...,...,...,...,...
55365,943,Reality Bites (1994),"[age-22, gender-M, occupation-student]","[release_date-Jan-1994, genre-comedy, genre-dr...",888640250
55366,943,Billy Madison (1995),"[age-22, gender-M, occupation-student]","[release_date-Jan-1995, genre-comedy]",888640251
55367,943,Jerry Maguire (1996),"[age-22, gender-M, occupation-student]","[release_date-Dec-1996, genre-drama, genre-rom...",888692413
55368,943,Willy Wonka and the Chocolate Factory (1971),"[age-22, gender-M, occupation-student]","[release_date-Jan-1971, genre-adventure, genre...",888692699


In [164]:
# 全ユーザーの最新1件をtestとして使用
df_test_indices = df.groupby("user_id")
df_test = df.loc[df_test_indices['timestamp'].idxmax(),:]

In [165]:
df_test

,user_id,movie_title,user_features,item_features,timestamp
161,1,When the Cats Away (Chacun cherche son chat) (...,"[age-24, gender-M, occupation-technician]","[release_date-Jun-1997, genre-comedy, genre-ro...",889751712
201,2,As Good As It Gets (1997),"[age-53, gender-F, occupation-other]","[release_date-Dec-1997, genre-comedy, genre-dr...",888979693
214,3,Return of the Jedi (1983),"[age-23, gender-M, occupation-writer]","[release_date-Mar-1997, genre-action, genre-ad...",889237482
235,4,Seven (Se7en) (1995),"[age-24, gender-M, occupation-technician]","[release_date-Jan-1995, genre-crime, genre-thr...",892004520
292,5,Rumble in the Bronx (1995),"[age-33, gender-F, occupation-other]","[release_date-Feb-1996, genre-action, genre-ad...",879198229
...,...,...,...,...,...
55132,939,"Associate, The (1996)","[age-26, gender-F, occupation-student]","[release_date-Oct-1996, genre-comedy]",880262090
55189,940,As Good As It Gets (1997),"[age-32, gender-M, occupation-administrator]","[release_date-Dec-1997, genre-comedy, genre-dr...",889480582
55206,941,Toy Story (1995),"[age-20, gender-M, occupation-student]","[release_date-Jan-1995, genre-animation, genre...",875049144
55272,942,Somewhere in Time (1980),"[age-48, gender-F, occupation-librarian]","[release_date-Jan-1980, genre-drama, genre-rom...",891283517


In [169]:
df_test["test_flag"] = 1
df_train = pd.merge(df, df_test[["test_flag"]], how="left", left_index=True, right_index=True)
df_train = df_train[df_train["test_flag"].isnull()].drop(columns=["test_flag"])

In [170]:
# user, item featureは全て既知として、既存のレコード全てからマスタを作成
uq_users = np.unique(df.user_id.values)
uq_items = np.unique(df.movie_title.values)
uq_user_features = np.unique(np.array(flatten_sequences(list(df.user_features.values))))
uq_item_features = np.unique(np.array(flatten_sequences(list(df.item_features.values))))

In [171]:
dataset = Dataset()
dataset.fit(uq_users, uq_items, item_features=uq_item_features, user_features=uq_user_features)

In [172]:
df_user_features = df[["user_id","user_features"]].drop_duplicates(subset='user_id').set_index('user_id')
df_item_features = df[["movie_title","item_features"]].drop_duplicates(subset='movie_title').set_index('movie_title')

In [174]:
# build interaction matrix
df_train_interactions = df_train[["user_id", "movie_title"]].drop_duplicates()
train_interactions = list(df_train_interactions.itertuples(index=False, name=None))
train_interactions, _ = dataset.build_interactions(train_interactions)

In [217]:
# test interaction matrix
df_test_interactions = df_test[["user_id", "movie_title"]].drop_duplicates()
test_interactions = list(df_test_interactions.itertuples(index=False, name=None))
test_interactions, _ = dataset.build_interactions(test_interactions)

In [176]:
# build user_feature matrix
user_features = list(df_user_features.itertuples(index=True, name=None))  # (user_id, [feature1, feature2, ...])
user_features = dataset.build_user_features(user_features)

# build item_feature matrix
item_features = list(df_item_features.itertuples(index=True, name=None))  # (item_id, [feature1, feature2, ...])
item_features = dataset.build_item_features(item_features)

In [177]:
# get mappings
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [178]:
from lightfm import LightFM

model = LightFM(loss='warp')
model.fit(train_interactions, item_features=item_features, user_features=user_features, epochs=100)

In [179]:
# user_idに対して、全アイテムへの予測値を格納したdfを返す
def predict_all_items_by_user_id(user_id):
    user_index = user_id_map[user_id]
    predicted = model.predict(user_ids=user_index, item_ids=np.array(range(0,len(item_id_map))))
    return make_predict_df(predicted)    
    
def make_predict_df(predicted):
    df_predicted = pd.DataFrame.from_dict(item_id_map, orient='index').rename(columns={0: "item_index"}).sort_values(by="item_index")
    df_predicted["score"] = predicted
    df_predicted = df_predicted.sort_values(by="score", ascending=False)
    return df_predicted[["score"]]

In [256]:
df_train[df_train["user_id"] == 873]

,user_id,movie_title,user_features,item_features,timestamp
50913,873,Rosewood (1997),"[age-48, gender-F, occupation-administrator]","[release_date-Feb-1997, genre-drama]",891392177
50914,873,Titanic (1997),"[age-48, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-action, genre-dr...",891392177
50915,873,Air Force One (1997),"[age-48, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-action, genre-th...",891392238
50916,873,Liar Liar (1997),"[age-48, gender-F, occupation-administrator]","[release_date-Mar-1997, genre-comedy]",891392303
50917,873,G.I. Jane (1997),"[age-48, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-action, genre-dr...",891392656
50918,873,"Man Who Knew Too Little, The (1997)","[age-48, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-comedy, genre-my...",891392698


In [181]:
predict_all_items_by_user_id(873)

,score
"Saint, The (1997)",3.641575
"Day the Earth Stood Still, The (1951)",3.111329
Devil in a Blue Dress (1995),3.071561
Congo (1995),3.038895
G.I. Jane (1997),2.924734
...,...
Down by Law (1986),-6.842842
2001: A Space Odyssey (1968),-7.080935
"Goofy Movie, A (1995)",-7.104146
Tomorrow Never Dies (1997),-7.716439


In [182]:
df[df["user_id"] == 222]

,user_id,movie_title,user_features,item_features,timestamp
12180,222,Chasing Amy (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Jan-1997, genre-drama, genre-rom...",877562748
12181,222,Contact (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Jul-1997, genre-drama, genre-sci...",877562748
12182,222,Conspiracy Theory (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Aug-1997, genre-action, genre-my...",877562772
12183,222,Air Force One (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Jan-1997, genre-action, genre-th...",877562795
12184,222,"Game, The (1997)","[age-29, gender-M, occupation-programmer]","[release_date-Jan-1997, genre-mystery, genre-t...",877562819
...,...,...,...,...,...
12323,222,Titanic (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Jan-1997, genre-action, genre-dr...",883814858
12324,222,Amistad (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Dec-1997, genre-drama]",883815120
12325,222,Scream (1996),"[age-29, gender-M, occupation-programmer]","[release_date-Dec-1996, genre-horror, genre-th...",883815252
12326,222,Scream 2 (1997),"[age-29, gender-M, occupation-programmer]","[release_date-Jan-1997, genre-horror, genre-th...",883815361


In [183]:
predict_all_items_by_user_id(222)

,score
"Saint, The (1997)",0.891780
Aladdin (1992),0.653654
This Is Spinal Tap (1984),-0.006518
Halloween: The Curse of Michael Myers (1995),-0.220789
Radioland Murders (1994),-0.397483
...,...
Disclosure (1994),-18.671017
That Old Feeling (1997),-19.250801
Strictly Ballroom (1992),-19.288551
Tomorrow Never Dies (1997),-21.962906


In [184]:
df[df["user_id"] == 514]

,user_id,movie_title,user_features,item_features,timestamp
32512,514,Contact (1997),"[age-27, gender-M, occupation-programmer]","[release_date-Jul-1997, genre-drama, genre-sci...",875308674
32513,514,"Wrong Trousers, The (1993)","[age-27, gender-M, occupation-programmer]","[release_date-Jan-1993, genre-animation, genre...",875308734
32514,514,12 Angry Men (1957),"[age-27, gender-M, occupation-programmer]","[release_date-Jan-1957, genre-drama]",875308925
32515,514,Monty Python and the Holy Grail (1974),"[age-27, gender-M, occupation-programmer]","[release_date-Jan-1974, genre-comedy]",875308925
32516,514,Emma (1996),"[age-27, gender-M, occupation-programmer]","[release_date-Aug-1996, genre-drama, genre-rom...",875309231
...,...,...,...,...,...
32645,514,Nell (1994),"[age-27, gender-M, occupation-programmer]","[release_date-Jan-1994, genre-drama]",886189841
32646,514,Six Degrees of Separation (1993),"[age-27, gender-M, occupation-programmer]","[release_date-Jan-1993, genre-drama, genre-mys...",886189893
32647,514,"Hunt for Red October, The (1990)","[age-27, gender-M, occupation-programmer]","[release_date-Jan-1990, genre-action, genre-th...",886190600
32648,514,Crimson Tide (1995),"[age-27, gender-M, occupation-programmer]","[release_date-Jan-1995, genre-drama, genre-thr...",886190665


In [185]:
predict_all_items_by_user_id(514)

,score
"Quick and the Dead, The (1995)",0.634016
Warriors of Virtue (1997),0.534648
Renaissance Man (1994),-0.048955
"Transformers: The Movie, The (1986)",-0.727365
Apollo 13 (1995),-1.508090
...,...
Star Wars (1977),-17.473375
Rob Roy (1995),-17.639675
"Crying Game, The (1992)",-17.722736
Young Guns II (1990),-17.804255


In [186]:
df[df["user_id"] == 400]

,user_id,movie_title,user_features,item_features,timestamp
25119,400,"English Patient, The (1996)","[age-33, gender-F, occupation-administrator]","[release_date-Nov-1996, genre-drama, genre-rom...",885676230
25120,400,"Full Monty, The (1997)","[age-33, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-comedy]",885676230
25121,400,Air Force One (1997),"[age-33, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-action, genre-th...",885676230
25122,400,Contact (1997),"[age-33, gender-F, occupation-administrator]","[release_date-Jul-1997, genre-drama, genre-sci...",885676316
25123,400,Titanic (1997),"[age-33, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-action, genre-dr...",885676316
25124,400,Scream (1996),"[age-33, gender-F, occupation-administrator]","[release_date-Dec-1996, genre-horror, genre-th...",885676365
25125,400,In & Out (1997),"[age-33, gender-F, occupation-administrator]","[release_date-Sep-1997, genre-comedy]",885676411
25126,400,"MatchMaker, The (1997)","[age-33, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-comedy, genre-ro...",885676452
25127,400,Mother (1996),"[age-33, gender-F, occupation-administrator]","[release_date-Dec-1996, genre-comedy]",885676452
25128,400,Scream 2 (1997),"[age-33, gender-F, occupation-administrator]","[release_date-Jan-1997, genre-horror, genre-th...",885676452


In [187]:
predict_all_items_by_user_id(400)

,score
"Saint, The (1997)",2.686638
First Knight (1995),2.311322
Congo (1995),2.285851
Air Force One (1997),2.110141
Devil in a Blue Dress (1995),1.974908
...,...
Laura (1944),-7.261855
2001: A Space Odyssey (1968),-7.323692
Down by Law (1986),-7.452860
Tomorrow Never Dies (1997),-8.184082


In [257]:
# unknownなuser_idに対して、user_featureを使用して予測する
# 参照： https://github.com/lyst/lightfm/issues/210

# predictにおいてuser_features, item_featuresが渡されたとき、user_id, item_idはuser_features, item_featuresにおけるindexを意味するため、以下のような処理になる
def predict_all_items_by_user_features(user_features):
    # user_featuresをmatrixに変換
    col = np.array([user_feature_map[key] for key in unknown_user_features])
    row = np.repeat(0, len(unknown_user_features))
    data = np.repeat(1, len(unknown_user_features))
    user_features = csr_matrix((data, (row, col)), shape=(1, len(user_feature_map)))
    
    # predict 
    predicted = model.predict(user_ids=0, item_ids=np.array(range(0,len(item_id_map))), user_features=user_features)
    return make_predict_df(predicted)

unknown_user_features = ["age-48", "gender-F", "occupation-administrator"]
predict_all_items_by_user_features(unknown_user_features)

,score
Warriors of Virtue (1997),-136.994888
"Transformers: The Movie, The (1986)",-139.079468
"Quick and the Dead, The (1995)",-139.177170
Renaissance Man (1994),-141.065994
Glory (1989),-142.129562
...,...
Young Guns II (1990),-228.544098
"Peacemaker, The (1997)",-229.503708
Shall We Dance? (1937),-232.021194
"Crying Game, The (1992)",-232.496429


In [110]:
# predict rank
# https://github.com/lyst/lightfm/issues/190
model.predict_rank(test_interactions=test_interactions, user_features=user_features, item_features=item_features)

In [205]:
# evaluation
from lightfm import evaluation

precisions = evaluation.precision_at_k(model, k=4, test_interactions=test_interactions, user_features=user_features, item_features=item_features)
recalls = evaluation.recall_at_k(model, k=4, test_interactions=test_interactions, user_features=user_features, item_features=item_features)
aucs = evaluation.auc_score(model, test_interactions=test_interactions, user_features=user_features, item_features=item_features)

In [209]:
recalls.mean()

0.0031847133757961785

In [230]:
# embeddingを取得する

user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

# user
user_embeddings = model.user_embeddings
mapped_user_embeddings = {name: user_embeddings[index] for name, index in user_feature_map.items()}
print("Embedding: gender-F")
print(mapped_user_embeddings["gender-F"])

# item
item_embeddings = model.item_embeddings
mapped_item_embeddings = {name: item_embeddings[index] for name, index in item_feature_map.items()}
print("Embedding: MatchMaker, The (1997)")
print(mapped_item_embeddings["MatchMaker, The (1997)"])

Embedding: gender-F
[-2.3874125   0.9395195  -2.2523065  -6.961481   -0.7835085   4.270282
 -0.47744742  0.6703578   6.288997    5.106352  ]
Embedding: MatchMaker, The (1997)
[ 0.48336646  0.1213533   0.6352143   0.08677411  0.5721241  -0.5955509
  0.6072041  -0.3372576  -0.7195242  -0.93152696]


In [254]:
def create_matrix_with_user_features(user_features):
    # csr_matrixに変換
    col = np.array([user_feature_map[key] for key in user_features])
    row = np.repeat(0, len(user_features))
    data = np.repeat(1, len(user_features))
    user_matrix = csr_matrix((data, (row, col)), shape=(1, len(user_feature_map)))
    return user_matrix

user_features = ["age-33", "gender-F", "occupation-administrator"]
user_matrix = create_matrix_with_user_features(user_features)

bias, embeddings = model.get_user_representations(features=user_matrix)

print(f"bias: {bias}")
print(f"embedding: {embeddings[0]}")

bias: [-195.15112]
embedding: [-4.9082627 -1.2549659 -4.376274  -8.901307  -2.6249416  4.056341
 -1.5262341  2.209255   7.078497   6.1518583]


In [239]:
bias

array([-0.15018658], dtype=float32)

In [240]:
embedding

array([ 0.11230547, -0.04001278, -0.29221743,  0.43767428,  0.13862383,
        0.3692265 ,  0.24970183,  0.17777517,  0.46091297,  0.04363978],
      dtype=float32)

In [264]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

print(f"index of user_feature age-48: {user_feature_map['age-48']}")
print(f"index of item_feature Full Monty, The (1997): {item_feature_map['Full Monty, The (1997)']}")

index of user_feature age-48: 979
index of item_feature Full Monty, The (1997): 498


In [275]:
dataset = Dataset()

dataset.fit([1, 2], uq_items, user_features=["age"])
user_features = dataset.build_user_features([[1, {"age": 20}], [2, {"age": 10}]], normalize=False)  # [[user_id, {feature1: weight, ...}, ...], ...])

In [276]:
user_features.toarray()

array([[0.04761905, 0.        , 0.95238096],
       [0.        , 0.09090909, 0.90909094]], dtype=float32)